In [ ]:
\l ../automl.q
.aml.loadfile`:init.q

In [ ]:
/ original table
n:100000
show t:([]`date$asc n?til 50;n?`A`B`C;n?1f;n?1f;asc n?1f;asc n?1000;n?1000)
show y:asc 50?til 6

In [ ]:
/ prepare data
t:.aml.i.symencode[t;10]
t^:.ml.stdscaler select x1,x2,x3,x4 from t

In [ ]:
/ keep a holdout set for testing
tts:.ml.traintestsplit[t;y;.2]
show holdout:-2#tts

In [ ]:
/ training and validation sets
show x:.ml.traintestsplit[;;.2]. tts`xtrain`ytrain

In [ ]:
/ import and train a random forest
.ml.mattab:{flip value flip x}
rfc:.p.import[`sklearn.ensemble][`:RandomForestClassifier][]
rfc[`:fit][.ml.mattab x`xtrain;x`ytrain];

In [ ]:
/ validate model
p:rfc[`:predict][.ml.mattab x`xtest]`;
.ml.accuracy[p;x`ytest]

In [ ]:
plt:.p.import`matplotlib.pyplot
.ml.mattab:{flip value flip x}
.ml.shuffle:{@[x;y;@;]neg[n]?n:count x}
.aml.rerun:{[m;x;y;c;f]
 x:.ml.shuffle[x;c];
 p:m[`:predict][.ml.mattab x]`;
 f[p;y]}
.aml.impact:{asc y!s%max s:$[z~`desc;1-;]$[any 0>x;.ml.minmaxscaler;]x}
.aml.impactplot:{[r;m]
 sub:plt[`:subplots][];
 fig:sub[@;0];ax:sub[@;1];
 ax[`:barh][n:til count r;value r;`align pykw`center];
 ax[`:set_yticks][n];
 ax[`:set_yticklabels]key r;
 ax[`:set_title]"Feature Impact: ",string m;
 ax[`:set_ylabel]"Columns";
 ax[`:set_xlabel]"Relative feature impact";
 /plt[`:savefig]sv["_";string(m;.z.Z)],".png";}
 plt[`:show][];}

In [ ]:
show s:.aml.rerun[rfc;holdout`xtest;holdout`ytest;;.ml.rmse]each c:cols t

In [ ]:
show r:.aml.impact[s;c;`asc]

In [ ]:
.aml.impactplot[r;`RandomForestClassifier]

---

Implement with pipeline

---

In [1]:
\l ../automl.q
.aml.loadfile`:init.q

In [5]:
n:10000
x:([]n?20;n?100f;asc n?100f)
yr:asc n?100f / regression
yc:20?0b       / binary classification
ym:n?`a`b`c   / multi classification
k:3           / number of folds
p:.2          / percentage of holdout
n:1           / number of repetitions
score_func:.ml.xv.fitpredict

In [6]:
tb:x
tgt:yc
mdls:.aml.models[`class;yc]
p:(::)
typ:`fresh

In [7]:
show dict:.aml.i.updparam[tb;p;typ];

aggcols| `x
params | (+(,`f)!,`absenergy`abssumchange`aggautocorr`augfuller`count`countab..
xv     | `kfsplit
prf    | {[f;p;d]($[-7h~type p;f[d;p];@[.[f[][p]`:fit;d 0]`:predict;d[1]0]`];..
scf    | `class`reg!`.ml.accuracy`.ml.mse
k      | 5
seed   | 42


In [8]:
system "S ",string s:dict`seed

In [9]:
show tb:.aml.i.autotype[tb;typ;dict]

x  x1       x2         
-----------------------
2  29.0322  0.002634036
12 72.30934 0.01441627 
0  83.75376 0.03936859 
2  84.79524 0.04090026 
1  19.42908 0.05090393 
14 19.1311  0.05482773 
2  25.85555 0.05538564 
5  25.51313 0.06610537 
7  93.41773 0.06665906 
6  60.71418 0.07138648 
15 58.83288 0.07692813 
6  86.58619 0.07980864 
6  30.96675 0.08175338 
16 37.79346 0.1002563  
2  85.73056 0.10602    
18 72.96694 0.1130171  
13 28.54082 0.1199678  
5  37.8652  0.1289898  
8  35.64297 0.1304162  
9  51.21478 0.1401179  
..


In [10]:
show tb:.aml.preproc[tb;tgt;typ;dict];

x  x1       x2         
-----------------------
2  29.0322  0.002634036
12 72.30934 0.01441627 
0  83.75376 0.03936859 
2  84.79524 0.04090026 
1  19.42908 0.05090393 
14 19.1311  0.05482773 
2  25.85555 0.05538564 
5  25.51313 0.06610537 
7  93.41773 0.06665906 
6  60.71418 0.07138648 
15 58.83288 0.07692813 
6  86.58619 0.07980864 
6  30.96675 0.08175338 
16 37.79346 0.1002563  
2  85.73056 0.10602    
18 72.96694 0.1130171  
13 28.54082 0.1199678  
5  37.8652  0.1289898  
8  35.64297 0.1304162  
9  51.21478 0.1401179  
..


In [11]:
show tb:.aml.freshcreate[tb;dict]

/home/deanna/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


x1_absenergy x1_abssumchange x1_count x1_countabovemean x1_countbelowmean x1_..
-----------------------------------------------------------------------------..
1670396      15856.66        490      247               243               0.6..
1635423      17395.65        491      250               241               0.0..
1732467      17433.42        506      253               253               0.3..
1693819      16076.21        477      241               236               0.4..
1561069      15663.79        471      228               243               0.8..
1577929      15685.5         490      241               249               0.1..
1622323      15660.2         497      245               252               0.2..
1667355      18606.42        520      255               265               0.2..
1670311      15641.76        494      246               248               0.7..
1666780      16641.52        512      258               254               0.6..
1523024      14457.78        472      22

In [13]:
/.aml.runmodels
if[11h~type tgt;tgt:![dy;til count dy:distinct tgt]tgt]

In [17]:
.aml.runexample

{[tb;tgt;typ;mdls;p]
 dict:i.updparam[tb;p;typ];
 system "S ",string s:dict`seed;
 tb:i.autotype[tb;typ;dict];
 tb:preproc[tb;tgt;typ;dict];
 -1"\nData preprocessing completed, starting feature creation\n";
 tb:freshcreate[tb;dict];
 -1"Feature creation completed, starting initial model selection - allow time..
 -1"Total features created = ",string[count 1_cols tb],"\n";
 r:runmodels[flip value flip tb;tgt;mdls;dict];
 -1"A ranking of the best models has now been completed, continuing to the ne..
 r}


In [18]:
.aml.runmodels

{[x;y;m;d]
 system"S ",string s:d`seed;
 if[11h~type y;y:![dy;til count dy:distinct y]y];
 r:.ml.gs.seed[x;y;d]'[m];
 fn:$[`reg in distinct m`typ;d[`scf]`reg;d[`scf]`class];  / scoring function ..
 sco:i.txtparse[`score;"/code/mdl_def/"];	          / sco = score ordering tab
 get[string first sco fn]m[`model]!{first avg x}each get[fn].''r}


In [20]:
dict

aggcols| `x
params | (+(,`f)!,`absenergy`abssumchange`aggautocorr`augfuller`count`countab..
xv     | `kfsplit
prf    | {[f;p;d]($[-7h~type p;f[d;p];@[.[f[][p]`:fit;d 0]`:predict;d[1]0]`];..
scf    | `class`reg!`.ml.accuracy`.ml.mse
k      | 5
seed   | 42


In [21]:
.ml.gs.seed

{[x;y;d;m]
 b:m[`lib]~`sklearn;
 s:$[a:m[`seed]~`seed;$[b;enlist[`random_state]!enlist d`seed;d`seed];::];
 $[a&b;first value get[` sv`.ml.gs,d`xv][d`k;1;x;y;d[`prf]m`minit;s;0];
   get[` sv`.ml.xv,d`xv][d`k;1;x;y;d[`prf][m`minit;s]]]}


In [18]:
r:.ml.gs.seed[tb;tgt;dict]'[mdls] //???

[0;31mcall: float() argument must be a string or a number, not 'dict'[0m: [0;31mcall: float() argument must be a string or a number, not 'dict'[0m

In [ ]:
tb:t;
tgt:y;
typ:`fresh
mdls:.aml.models[`class;y]
p:(::)

In [ ]:
dict:.aml.i.updparam[tb;p;typ]

In [ ]:
runexample:{[tb;tgt;typ;mdls;p]
 dict:i.updparam[tb;p;typ];
 system "S ",string s:dict`seed;
 tb:i.autotype[tb;typ;dict];
 tb:preproc[tb;tgt;typ;dict];
 -1"\nData preprocessing completed, starting feature creation\n";
 tb:freshcreate[tb;dict];
 -1"Feature creation completed, starting initial model selection - allow time for large datasets\n";
 -1"Total features created = ",string[count 1_cols tb],"\n";
 r:runmodels[flip value flip tb;tgt;mdls;dict];
 -1"A ranking of the best models has now been completed, continuing to the next step\n";
 r}